# Embeddings

## Generating an initial embedding

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
print("Libraries imported successfully")

In [ ]:
# Load an embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")
print(f"Model produces {model.get_sentence_embedding_dimensions()} dimensional embeddings")

In [ ]:
# Generating embedding
text = "The cat sat on the mat"

embedding = model.encode(text)
print(f"Embedding shape: {embedding.shape}")
print(f"\nFirst 10 embedding values:\n{embedding[:10]}")